In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman MANI File ETL

### Config

In [2]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20240923'
# pro_file_name = 'ACO Builder MSSP Prospective Explorer v2025.3.txt'
# retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2025.3.txt'
table_prefix = 'milliman_mani'

### DataTables

In [3]:
file_name = 'Medicare Advantage Network Intelligence (MANI) CY2021 Data Tables - v2024.1.0 - Nationwide.csv'
year = 2021
mani_data_tables_2020_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='utf-8-sig', low_memory=False)
# display(mani_data_tables_2020_df.head())
mani_data_tables_2020_df = cb_utils.df_format_columns(mani_data_tables_2020_df)
mani_data_tables_2020_df.head()

,year,aco_builder_id,tin_ccn_type,tin_ccn_id,provider_name,pac_id,st_msa,carrier,py,risk_score_mmwtd,risk_score_mmwtd_region,ma_benchmark_rate,ma_bid_revenue,paid_atffsreimb,paid_atffsreimb_region,ip_admits,ip_admits_imputed,ip_admits_region,ip_days,ip_days_imputed,ip_days_region,snf_admits,snf_admits_imputed,snf_admits_region,snf_days,snf_days_imputed,snf_days_region,ed,ed_imputed,ed_region,phy_office,phy_office_imputed,phy_office_region,rad_adv_imag_op,rad_adv_imag_op_imputed,rad_adv_imag_op_region,rad_adv_imag_nonhosp,rad_adv_imag_nonhosp_imputed,rad_adv_imag_nonhosp_region,rad_gen_op,rad_gen_op_imputed,rad_gen_op_region,rad_gen_nonhosp,rad_gen_nonhosp_imputed,rad_gen_nonhosp_region,rad_therap_op,rad_therap_op_imputed,rad_therap_procs_op_region,rad_therap_nonhosp,rad_therap_nonhosp_imputed,rad_therap_nonhosp_region,prev_physicals,prev_physicals_imputed,prev_physicals_region,surgery_asc,surgery_asc_imputed,surgery_asc_region,surgery_op,surgery_op_imputed,surgery_op_region,part_b_rx_nonhosp,part_b_rx_nonhosp_imputed,part_b_rx_nonhosp_region,part_b_rx_op,part_b_rx_op_imputed,part_b_rx_op_region,lab_nonhosp,lab_nonhosp_imputed,lab_nonhosp_region,lab_op,lab_op_imputed,lab_op_region
0,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Blue Cross of Idaho Health Services, Inc.",55.91667,649.55506,640.83888,6.356744e+05,559424.56810,5.910014e+05,6.072972e+05,17.13547,17.13547,11.03589,86.84419,31.84419,66.06327,NaN,NaN,1.94593,NaN,NaN,41.71051,85.00000,NaN,30.43373,451.61960,4.61960,419.34695,45.0,NaN,41.71185,8.53922,8.53922,7.50197,66.00000,NaN,62.83984,44.00000,NaN,53.47546,NaN,NaN,4.70126,NaN,NaN,1.32729,20.74052,6.74052,28.79153,12.0,NaN,4.54270,13.63203,2.63203,30.05566,59.22386,3.22386,51.64468,2.00000,2.00000,63.85253,329.0,NaN,581.96436,89.0000,NaN,431.75975
1,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,CVS Health Corporation,2.00000,21.77081,22.32189,2.226602e+04,18505.30110,2.330050e+04,2.247332e+04,NaN,NaN,0.41505,NaN,NaN,1.43143,NaN,NaN,0.13481,NaN,NaN,3.50537,NaN,NaN,1.88892,2.77923,2.77923,9.99023,NaN,NaN,1.58716,NaN,NaN,0.29086,NaN,NaN,1.56775,NaN,NaN,0.53925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.89209,NaN,NaN,0.35294,NaN,NaN,1.71313,NaN,NaN,1.17224,NaN,NaN,1.53589,135.0,NaN,26.60156,NaN,NaN,14.80801
2,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,FFS,134.33333,1615.36737,1571.66835,1.588136e+06,0.00000,1.326359e+06,1.559343e+06,26.50984,14.50984,30.52918,138.10190,12.10190,159.59043,2.0,2.0,5.48214,40.0,NaN,157.26190,48.00000,NaN,61.11761,874.00000,NaN,1061.20297,104.0,NaN,117.30828,10.94227,10.94227,24.13127,77.00000,NaN,147.56760,49.66007,5.66007,130.44540,11.0,NaN,17.26432,NaN,NaN,5.89768,47.02285,2.02285,47.95079,1.0,1.0,11.09491,38.00000,11.00000,52.59237,96.00000,NaN,152.58745,19.96174,1.96174,162.99213,3083.0,NaN,1397.89254,277.0000,NaN,1152.79314
3,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,Humana Inc.,4.08333,64.41775,62.54246,6.242565e+04,56951.49747,6.156252e+04,4.705519e+04,1.00000,1.00000,0.92975,3.08190,3.08190,5.20234,NaN,NaN,0.22631,NaN,NaN,4.96909,1.14557,1.14557,1.89612,54.00000,NaN,31.47960,1.0,1.0,3.21802,3.06721,3.06721,0.95441,8.43888,8.43888,4.24309,1.85697,1.85697,3.80270,NaN,NaN,0.39524,NaN,NaN,0.23510,2.83722,2.83722,2.11295,NaN,NaN,0.37407,4.34297,4.34297,2.04491,1.00000,1.00000,3.75812,41.00000,1.00000,4.04710,110.0,NaN,38.83154,34.0000,NaN,43.90792
4,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,ID Other Carriers,2.00000,6.49522,7.23472,6.367185e+03,5406.79273,9.439423e+03,7.118980e+03,NaN,NaN,0.08200,NaN,NaN,0.38155,NaN,NaN,0.00529,NaN,NaN,0.10320,NaN,NaN,0.20384,9.57562,9.57562,7.51428,NaN,NaN,0.51544,NaN,NaN,0.15390,NaN,NaN,0.81409,1.00000,1.00000,1.24823,NaN,NaN,0.07928,NaN,NaN,0.05823,1.96687,1.96687,1.23962,NaN,NaN,0.19276,1.00000,1.00000,0.67145,1.00000,1.00000,0.83862,9.08134,9.08

In [4]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_mani_data_tables_2021_20240923


In [5]:
mani_data_tables_2020_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

352477

In [9]:
file_name = 'Medicare Advantage Network Intelligence (MANI) CY2022 Data Tables - v2024.1.0 - Nationwide.csv'
year = 2022
mani_data_tables_2021_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='latin', low_memory=False)
# display(mani_data_tables_2020_df.head())
mani_data_tables_2021_df = cb_utils.df_format_columns(mani_data_tables_2021_df)
mani_data_tables_2021_df.head()

,year,aco_builder_id,tin_ccn_type,tin_ccn_id,provider_name,pac_id,st_msa,carrier,py,risk_score_mmwtd,risk_score_mmwtd_region,ma_benchmark_rate,ma_bid_revenue,paid_atffsreimb,paid_atffsreimb_region,ip_admits,ip_admits_imputed,ip_admits_region,ip_days,ip_days_imputed,ip_days_region,snf_admits,snf_admits_imputed,snf_admits_region,snf_days,snf_days_imputed,snf_days_region,ed,ed_imputed,ed_region,phy_office,phy_office_imputed,phy_office_region,rad_adv_imag_op,rad_adv_imag_op_imputed,rad_adv_imag_op_region,rad_adv_imag_nonhosp,rad_adv_imag_nonhosp_imputed,rad_adv_imag_nonhosp_region,rad_gen_op,rad_gen_op_imputed,rad_gen_op_region,rad_gen_nonhosp,rad_gen_nonhosp_imputed,rad_gen_nonhosp_region,rad_therap_op,rad_therap_op_imputed,rad_therap_op_region,rad_therap_nonhosp,rad_therap_nonhosp_imputed,rad_therap_nonhosp_region,prev_physicals,prev_physicals_imputed,prev_physicals_region,surgery_asc,surgery_asc_imputed,surgery_asc_region,surgery_op,surgery_op_imputed,surgery_op_region,part_b_rx_nonhosp,part_b_rx_nonhosp_imputed,part_b_rx_nonhosp_region,part_b_rx_op,part_b_rx_op_imputed,part_b_rx_op_region,lab_nonhosp,lab_nonhosp_imputed,lab_nonhosp_region,lab_op,lab_op_imputed,lab_op_region
0,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Blue Cross of Idaho Health Service, Inc.",65.08333,639.85391,636.02106,6.642034e+05,529726.06800,4.322393e+05,5.797445e+05,2.28381,2.28381,8.79512,14.00000,NaN,52.37031,NaN,NaN,1.77140,NaN,NaN,40.85389,14.64854,3.64854,31.87719,473.00000,NaN,465.48660,9.88028,9.88028,42.55615,6.98936,6.98936,9.56654,42.00000,NaN,54.21169,47.00000,NaN,67.57730,NaN,NaN,6.37519,23.0,NaN,3.87828,44.71640,9.71640,38.20341,7.95918,7.95918,6.75815,22.00000,NaN,33.91253,47.49284,3.49284,59.83047,22.0,2.0,58.99839,1187.00000,NaN,726.10291,162.00123,6.00123,418.98489
1,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,FFS,121.66667,1343.30501,1298.78617,1.397050e+06,0.00000,1.347886e+06,1.295010e+06,37.38080,20.38080,23.75523,295.01085,4.01085,128.45655,2.49247,2.49247,5.25985,68.0,NaN,129.31867,60.00000,NaN,55.77226,938.00000,NaN,903.83415,98.00000,NaN,82.54304,3.70142,3.70142,23.06506,86.00000,NaN,106.47740,70.00000,NaN,120.01398,3.73658,3.73658,15.21148,NaN,NaN,5.52193,36.73723,5.73723,47.19721,10.34100,10.34100,9.46332,30.33249,2.33249,43.68030,140.00000,NaN,130.08748,32.0,1.0,129.58221,2246.00000,NaN,1219.14329,457.00000,NaN,1017.01127
2,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,Humana Inc.,4.00000,57.05591,52.51653,5.917675e+04,47198.96697,9.234079e+04,4.642003e+04,1.46383,1.46383,0.82636,7.40467,7.40467,5.18964,NaN,NaN,0.18189,NaN,NaN,3.74707,6.84581,6.84581,2.50946,44.14938,9.14938,31.63660,6.14034,6.14034,2.97931,1.00000,1.00000,0.61816,3.77404,3.77404,3.95435,2.00000,2.00000,4.52106,NaN,NaN,0.14489,NaN,NaN,0.25646,2.87848,2.87848,1.99091,NaN,NaN,0.40974,5.78241,5.78241,2.49097,1.00000,1.00000,3.35319,24.0,NaN,5.60880,89.58968,4.58968,45.50537,14.27953,3.27953,39.71873
3,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,ID Other Carriers,1.00000,10.06988,9.18759,1.079159e+04,8607.29534,2.720179e+03,6.764767e+03,NaN,NaN,0.11525,NaN,NaN,0.54893,NaN,NaN,0.02072,NaN,NaN,0.35509,NaN,NaN,0.28848,3.38597,3.38597,6.44971,NaN,NaN,0.60461,NaN,NaN,0.13406,NaN,NaN,0.74396,NaN,NaN,0.80383,NaN,NaN,0.06456,NaN,NaN,0.05376,1.00000,1.00000,0.80896,NaN,NaN,0.09435,NaN,NaN,0.27690,1.00000,1.00000,0.85836,NaN,NaN,0.47189,12.00000,NaN,9.20215,NaN,NaN,4.15640
4,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Molina Healthcare, Inc.,",5.33333,125.73039,86.55200,1.306790e+05,104228.63500,1.062689e+04,9.071488e+04,NaN,NaN,1.64014,NaN,NaN,12.80186,NaN,NaN,0.49274,NaN,NaN,12.20898,2.40522,2.40522,6.57031,49.00000,11.00000,52.99219,1.00000,1.00000,5.82617,1.00000,1.00000,0.96692,1.68345,1.68345,8.41211,2.96085,2.96085,6.44434,NaN,NaN,0.34137,NaN,NaN,0.12643,

In [10]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_mani_data_tables_2022_20240923


In [11]:
mani_data_tables_2021_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

378684

# Next time

In [ ]:
data_table_yrs = [
    {yr, file_name, encoding}
]

In [ ]:
for {yr, file_name, encoding} in data_table_yrs:
    
    if encoding = None:
        encoding = 'latin'
        
    mani_data_tables_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding=encoding, low_memory=False)
    # display(mani_data_tables_2020_df.head())
    mani_data_tables_df = cb_utils.df_format_columns(mani_data_tables_df)
    conn = cb_utils.get_engine(source='msh_analytics')
    table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
    schema = 'raw'
    print(f'select * from {schema}.{table_name}')
    mani_data_tables_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'